Lonboard er et bibliotek for å vise kart i Jupyter notebooks.
De som har brukt Geopandas tidligere vet at det er mulig å vise frem kart i Jupyter med explore metoden på en GeoDataFrame, så hva er egentlig spesielt med Lonboard?

Lonboard er bygget på [deck.gl](https://deck.gl), et GPU akselerert, høytytende, kartvisualiseringsbibliotek for store data og Anywidget.
Navnet er et ordspill. Et «longboard», en type raskt skateboard, et «deck» er den delen av skateboardet du står på. «Lon» en mye brukt forkortelse for «longitude», lengdegrad.

Geopandas explore metode, og explore funksjonen i ssb-sgis pakken er bygget på Folium. 
Hver gang du skal vise frem et kart med Folium må kartdataene konverteres til et format som kalles Geojson, 
som så sendes ukomprimert fra Jupyter-serven til nettleseren. Dette gjør at du fort går tom for minne om du forsøker å vise et landsdekkene datasett, slik som grunnkretser, tettsteder, eller postnummerområder. 
Lonboard takler større mengder data ved å overføre data mellom serveren og nettleseren på Parquet-formatet istedenfor Geojson-formatet.

Hvor kommer Anywidget inn? Anywidget er et rammeverk for å lage widgets.

In [21]:
#| include: false
import geopandas as gpd
import lonboard
from lonboard import basemap
from mapclassify import greedy
from matplotlib import colormaps
import numpy as np


In [22]:
grunnkretser = gpd.read_file(
    "https://nedlasting.geonorge.no/geonorge/Basisdata/Grunnkretser/GML/Basisdata_34_Innlandet_25833_Grunnkretser_GML.zip",
    layer="Grunnkrets",
    engine="pyogrio",
    columns=["grunnkretsnummer", "grunnkretsnavn", "kommunenummer"],
)

grunnkretser.head()

,grunnkretsnavn,grunnkretsnummer,kommunenummer,geometry
0,Fagernes nord-Skrautvålsvegen,34510108,3451,"MULTIPOLYGON (((188273.260 6775295.340, 188136..."
1,Ilseng,34030804,3403,"MULTIPOLYGON (((294050.140 6743722.400, 294052..."
2,Sognefjell,34340110,3434,"MULTIPOLYGON (((124979.390 6858916.550, 124085..."
3,Nordråk,34470105,3447,"MULTIPOLYGON (((241913.330 6741294.510, 241567..."
4,Byre,34340201,3434,"MULTIPOLYGON (((174511.100 6874025.010, 174456..."


In [23]:
#| include: false
def omskaler(
    s,
    max_ut: float,
    min_ut: float,
    max_cutoff: float | None = None,
    min_cutoff: float | None = None,
):
    max_inn = max(s) if not max_cutoff else max_cutoff
    min_inn = min(s) if not min_cutoff else min_cutoff

    skalering = (min_ut - max_ut) / (min_inn - max_inn)
    s = (s - min_inn) * skalering + min_ut

    if max_cutoff and max_inn > max_cutoff:
        s = np.minimum(s, max_ut)

    if min_cutoff and min_inn < min_cutoff:
        s = np.maximum(s, min_ut)

    return s

In [24]:
#| include: false
#| warnings: false
color = greedy(grunnkretser, strategy="balanced", balance="centroid").map(
    colormaps["Set2"].colors.__getitem__
)
color = (np.stack(color.to_numpy()) * 255).astype(np.uint8)

c:\Users\rnl\.conda\envs\ssb\Lib\site-packages\mapclassify\greedy.py:304: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  sw = Queen.from_dataframe(gdf, silence_warnings=silence_warnings)


In [25]:
grunnkretser_wgs84 = grunnkretser.to_crs(4326)
layer = lonboard.PolygonLayer.from_geopandas(
    grunnkretser_wgs84,
    opacity=0.2,
    line_miter_limit=1,
    line_width_units = "pixels",
    get_fill_color=color,
    get_line_color=[255,255,255],
    auto_highlight=True,
)


In [26]:
#| include: false
store_grunnkretser = grunnkretser.loc[grunnkretser.area > 40_000_000]
tekststørrelse = omskaler(store_grunnkretser.area, 16, 12)

tekst = lonboard.experimental.TextLayer.from_geopandas(
    store_grunnkretser.set_geometry(store_grunnkretser.geometry.representative_point()).to_crs(4326),
    get_text=store_grunnkretser["grunnkretsnavn"],
    font_family="Helsinki",
    get_size=tekststørrelse,
    pickable=False,
    get_color=[255,255,255],
)

In [ ]:
#| output: false
kart = lonboard.Map(
    [layer, tekst],
    basemap_style=basemap.CartoBasemap.DarkMatterNoLabels,
    _height=500,
)

kart

In [28]:
#| echo: false
# Qaurto støtter selvfølgelig ikke Anywidget, 
# men vi kan bruke kompabilitetsfunksjonen as_html, 
# fjerner resultatet av forrige celle og later bare som om resultatet kommer derfra.
# kart.as_html()

![](mapselection.png)

In [30]:
if kart.selected_bounds:
    xmin, ymin, xmax, ymax = kart.selected_bounds
    utvalg = grunnkretser_wgs84.cx[xmin:xmax, ymin:ymax]
    print(f"Det er {len(utvalg)} grunnkretser i utvalget")

Det er 41 grunnkretser i utvalget
